A neuron with a sigmoid function for its activation.

In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def sigmoid(t):
  return(1/(1+np.exp(-t)))

In [3]:
def LIF(V, ref, I=1, C=1, R=40, V_th=10, abs_ref=2):
  # Input current
  # I = nA

  # Capacitance and leak resistance
  # C = nF
  # R = M ohms

  # I & F implementation dV/dt = - V/RC + I/C
  # Using h = 1 ms step size, Euler method

  # Absolute refractory period lasts abs_ref turns
  # V_th = mV spike threshold

  if not ref:
    V = V - (V/(R*C)) + (I/C)
  else:
    ref -= 1
    V = 0.2 * V_th # reset voltage
   
  if V > V_th:
    V = 50 # emit spike
    ref = abs_ref # set refractory counter
  
  return V, ref

In [62]:
class Neuron():
  def __init__(self, th = 0.5, function = sigmoid):
    self.function = function
    self.threshold = th
  
  def fire(self, pre_syn):
    t = np.sum(pre_syn)
    yeet = self.function(t)
    return (1 if yeet >= self.threshold else 0)

In [63]:
class Layer():
  def __init__(self, neurons=None, weights=None):
    self.neurons = neurons
    self.weights = weights

    assert len(self.neurons) <= len(self.weights)

  def get_layer(self):
    return self.neurons, self.weights

  def update_weights(self, new_weights):
    assert len(self.weights) == len(new_weights)
    self.weights = new_weights

In [64]:
class Network():
  def __init__(self, layers):
    self.network_neu = []
    self.network_wei = []
    self.layers = layers
    self.output = []
    for lay in layers:
      self.network_neu.append(lay.get_layer()[0])
      self.network_wei.append(lay.get_layer()[1])

  def get_network_neurons(self):
    return self.network_neu

  def get_network_weights(self):
    return self.network_wei
  
  def get_outputs(self):
    return self.output

  def update_weights(self, new_weights):
    assert len(self.network_wei) == len(new_weights)
    assert len(self.network_wei[0]) == len(new_weights[0])
    for i in range(len(new_weights)):
      self.layers[i].update_weights(new_weights[i])
      self.network_wei[i] = new_weights[i]
  
  def compute(self, input):
    assert len(input) == len(self.network_neu[0])**2
    n = len(self.network_neu[0])
    self.output = []
    for i in range(len(self.network_neu)): # layers
      cur_outs = []
      for j in range(n): # neurons
        if i == 0:
          neuron = self.network_neu[i][j]
          cur_outs.append(neuron.fire(self.network_wei[i][j]*input[j]))
        else:
          cur_wei = self.network_wei[i][j*n:j*n+n]
          cur_act = self.output[i-1][j*n:j*n+n]
          cur_outs.append(neuron.fire([a*b for a,b in zip(cur_wei, cur_act)]))
      self.output.append(cur_outs)
    

In [7]:
n1 = Neuron()
s1 = [-3, 2]
n1.fire(pre_syn=s1)

0

In [68]:
# Define network structure parameters
neurons = []
weights = []
layers = []
N_lay = 7 # number of layers
N_neu = 50 # number of neurons PER LAYER

# Build network contents
for i in range(N_lay):
  for j in range(N_neu):
    neurons.append(Neuron())
    for k in range(N_neu):
      weights.append(np.random.random()) # Fully connected, equal sized layers
  layers.append(Layer(neurons, weights))
  neurons = []
  weights = []

# Construct network
network = Network(layers)

In [66]:
input = []
for b in range(N_neu*N_neu):
  input.append(np.random.random())
network.compute(input)

In [18]:
network.get_network_weights()[0][8]

0.9490133563766802

In [71]:
sum(network.get_outputs()[-1])

IndexError: ignored